<a href="https://colab.research.google.com/github/opeyemijanet/TECH_4_DEV_PROJECT/blob/main/cash_flow_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all required libraries


# This installs:
# - Flask (for building the API)
# - Flask-CORS (for allowing browsers to call your API)
# - pandas (for working with data)

In [ ]:
print("📦 Installing ngrok...")
!pip install pyngrok -q
!pip install flask flask-cors pandas -q

📦 Installing ngrok...


In [ ]:
from flask import Flask, request, jsonify

In [ ]:
!pip install flask-cors
from flask_cors import CORS

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import json

In [ ]:
#create a flask app

In [ ]:
app = Flask(__name__)

In [ ]:
CORS(app) #allows anyone to call our API

In [ ]:
#The AI model, the cashflow prediction


    This function predicts if a business will run out of money
    
    HOW IT WORKS (Simple Algorithm):
    1. Calculate average daily expenses (how much they spend per day)
    2. Calculate average daily income (how much they earn per day)
    3. Calculate daily burn rate (are they losing or gaining money?)
    4. Predict when they'll hit zero (if they're losing money)
    
    INPUTS:
    - transactions: list of dictionaries with date, type, amount
    - current_balance: how much money they have right now
    
    OUTPUTS:
    - risk_level: "Stable", "Warning", or "Critical"
    - days_until_broke: how many days until money runs out
    - explanation: why we gave this prediction
    

In [ ]:
def predict_cashflow_risk(transactions, current_balance):
    # Convert transactions to pandas data frame
    df = pd.DataFrame(transactions)

    # Check if we have enough data
    if len(df) < 10:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "Need at least 10 transactions to make prediction",
            "transactions_provided": len(df),
            "minimum_required": 10
        }

    # Separate income from expense
    income_df = df[df["type"] == "income"]
    expense_df = df[df["type"] == "expense"]

    # Check if we have both income and expense
    if len(income_df) == 0:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "No income transactions found. Need both income and expenses."
        }

    if len(expense_df) == 0:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "No expense transactions found. Need both income and expenses."
        }

    # Calculate averages.
    avg_daily_income = income_df['amount'].mean()
    avg_daily_expenses = expense_df['amount'].mean()

    # Calculate how much the business is burning through cash each day
    # Negative number = making money
    # Positive number = losing money
    daily_burn_rate = avg_daily_expenses - avg_daily_income

    # calculate total income and expenses
    total_income = income_df["amount"].sum()
    total_expenses = expense_df["amount"].sum()
    net_cash_flow = total_income - total_expenses

    #predict days until broke
    if daily_burn_rate > 0:
        risk_level = "stable"
        days_until_broke = None
        message = "Your business is financially stable. Income exceeds expenses."
    else:
        if daily_burn_rate == 0:
            risk_level = "stable"
            days_until_broke = None
            message = "Your business is breaking even. Monitor regularly!"
        else: # Business is losing money (daily_burn_rate < 0)
            # Use abs(daily_burn_rate) to get a positive number of days until broke
            days_until_broke = int(current_balance / abs(daily_burn_rate))

            # determine risk level based on days remaining
            if days_until_broke <= 30:
                risk_level = "critical"
                message = f'urgent cash shortage predicted in {days_until_broke} days!'
            elif days_until_broke <= 60:
                risk_level = "warning"
                message = f'caution! cash runway is {days_until_broke} days. plan ahead!'
            else:
                risk_level = "OK"
                message = f'You have {days_until_broke} days of cash runway. Monitor regularly!'


    # Check BOTH transaction count AND days
df['date'] = pd.to_datetime(df['date'])
days_of_data = (df['date'].max() - df['date'].min()).days + 1

# Validation
if len(df) < 10:
    return error
if days_of_data < 7:
    return warning

# Confidence based on DAYS (primary) + Density (secondary)
if days_of_data >= 90:
    days_score = 1.0  # ✅ 90 DAYS!
elif days_of_data >= 60:
    days_score = 0.85
elif days_of_data >= 30:
    days_score = 0.7
else:
    days_score = 0.5

transactions_per_day = len(df) / days_of_data
if transactions_per_day >= 5:
    density_score = 1.0
else:
    density_score = 0.7

confidence = days_score * density_score


    #Generate detailed explanation
    explanation = {
        'summary': message,
        'reasoning': f'based on your spending patterns: you spend on an average of {avg_daily_expenses:,.0f} per day but earn {avg_daily_income:,.0f}',
        'key_metrics': {
            'current_balance': current_balance,
            'avg_daily_income': float(avg_daily_income),
            'avg_daily_expenses': float(avg_daily_expenses),
            'daily_burn_rate': round(daily_burn_rate, 2),
            'total_income': round(total_income, 2),
            'total_expenses': round(total_expenses, 2),
            'net_cash_flow': round(net_cash_flow, 2),
        },
        'data_quality': {
            'transaction_count': int(transaction_count),
            'income_count': int(len(income_df)),
            'expense_count': len(expense_df),
            'confidence_score': confidence_level
        },
        'recommendations': generate_recommendations(risk_level, days_until_broke, current_balance) # Placeholder
    }

    #return the prediction
    return{
        'status': 'success',
        'risk_level': risk_level,
        'days_until_broke': int(days_until_broke) if days_until_broke is not None else None,
        'confidence_score': confidence_level,
        'explanation': explanation,
        'timestamp': datetime.now().isoformat()
    }

#Generate actionable recommendations based on the risk level
#This tells the user WHAT TO DO based on the prediction
def generate_recommendations(risk_level, days_until_broke, current_balance):
  recommendations = []
  if risk_level == 'critical':
    recommendations = [
        {
        'priority': 1,
        'action': 'Immediately reduce non-essential expenses',
        'impact': f'could extend runway by {days_until_broke} days'
        },
        {
        'priority': 2,
        'action': f'follow up on all pending payments urgently',
        'impact': 'Bring in cash faster'
        },
        {
        'priority': 3,
        'action': f'consider emergency short term financing',
        'impact': f'could extend runway by {days_until_broke} days'
        },
    ]
  elif risk_level == 'warning':
    recommendations = [
       {
        'priority': 1,
        'action': 'Review and reduce discretionary spending',
        'impact': 'Improve cash runway'
        },
        {
        'priority': 2,
        'action': 'Speed up collection of receivables',
        'impact': 'Improve cash inflow'
        },
        {
        'priority': 3,
        'action': 'Plan for additional revenue sources',
        'impact': 'Strengthen financial position'
        }
    ]
  else:
    recommendations =[
        {'priority': 1,
        'action': 'Continue monitoring cashflow weekly',
        'impact': 'Early detection of potential issues'
        },
        {
        'priority': 2,
        'action': 'Build emergency cash reserve',
        'impact': 'Protect against unexpected expenses'
        }
    ]
    return recommendations

In [ ]:
# ============================================
# API ENDPOINTS (THE "MENU" OF OUR RESTAURANT)
# ============================================

# --------------------------------------------
# ENDPOINT 1: Home/Welcome
# --------------------------------------------
@app.route('/', methods=['GET'])
def home():
    """
    This is the home page of the API
    When someone visits the main URL, they see this

    URL: http://your-api.com/
    """
    return jsonify({
        'message': 'Zero Hunger - Cashflow Prediction API',
        'version': '1.0',
        'status': 'running',
        'endpoints': {
            '/': 'This welcome message',
            '/api/predict-cashflow': 'Predict cashflow risk (POST)',
            '/api/health': 'Check if API is working'
        },
        'usage': {
            'endpoint': '/api/predict-cashflow',
            'method': 'POST',
            'required_fields': ['transactions', 'current_balance'],
            'example': {
                'transactions': [
                    {'date': '2026-01-15', 'type': 'income', 'amount': 50000},
                    {'date': '2026-01-15', 'type': 'expense', 'amount': 20000}
                ],
                'current_balance': 250000
            }
        }
    })




In [ ]:
# STEP 5: API Endpoints
# The home route is already defined in cell Hs5OK_rb0TqM, so it's removed here to avoid duplication.

@app.route('/api/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'message': 'API is running in Google Colab!'
    })

@app.route('/api/predict-cashflow', methods=['POST'])
def predict():
    try:
        data = request.json

        if not data or 'transactions' not in data or 'current_balance' not in data:
            return jsonify({
                'status': 'error',
                'message': 'Missing required fields: transactions and current_balance'
            }), 400

        result = predict_cashflow_risk(data['transactions'], data['current_balance'])

        if result['status'] == 'error':
            return jsonify(result), 400

        return jsonify(result), 200

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 500

In [ ]:
import threading
from pyngrok import ngrok
import os

# Kill any processes using port 5000 to prevent 'Address already in use' errors
!kill $(lsof -t -i:5000)

# STEP 6: Run with ngrok (creates public URL)
def run_app():
    """Run Flask in background thread"""
    # Use a different port if 5000 is persistently busy, e.g., app.run(port=5001)
    app.run(port=5000)

# Start Flask in background
thread = threading.Thread(target=run_app)
thread.start()

# Authenticate ngrok
# IMPORTANT: Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok authtoken.
# You can get one from https://dashboard.ngrok.com/get-started/your-authtoken
# Example: ngrok.set_auth_token("2P7y8...your_token...X9uH")
ngrok.set_auth_token("39J0ALzxdiR0pBc4oGnzTU4BVNx_33ePBd6Sz7pYyZvpU1nxs")

# Create ngrok tunnel
public_url = ngrok.connect(5000)

print("=" * 60)
print("🚀 API IS NOW RUNNING!")
print("=" * 60)
print(f"\n✅ Public URL: {public_url}")
print(f"\nYou can now:")
print(f"1. Click the URL above to test in browser")
print(f"2. Call API at: {public_url}/api/predict-cashflow")
print(f"3. Use this URL from anywhere (not just Colab)")
print("\n⚠️  Important: This URL will stop working when you stop this cell")
print("=" * 60)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


🚀 API IS NOW RUNNING!

✅ Public URL: NgrokTunnel: "https://sacculate-wormish-regine.ngrok-free.dev" -> "http://localhost:5000"

You can now:
1. Click the URL above to test in browser
2. Call API at: NgrokTunnel: "https://sacculate-wormish-regine.ngrok-free.dev" -> "http://localhost:5000"/api/predict-cashflow
3. Use this URL from anywhere (not just Colab)

⚠️  Important: This URL will stop working when you stop this cell
